In [1]:
import numpy as np
import joblib
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
import nltk
import pickle

In [2]:
# Descargar recursos de NLTK
nltk.download('vader_lexicon')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [3]:
# Definir la capa de atencion personalizada
class AttentionLayer(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W = self.add_weight(name="att_weight", shape=(input_shape[-1], 1), initializer="normal")
        self.b = self.add_weight(name="att_bias", shape=(input_shape[1], 1), initializer="zeros")
        super(AttentionLayer, self).build(input_shape)

    def call(self, x):
        e = tf.keras.activations.tanh(tf.keras.backend.dot(x, self.W) + self.b)
        a = tf.keras.activations.softmax(e, axis=1)
        output = x * a
        return tf.keras.backend.sum(output, axis=1)

In [4]:
# Cargar modelo de NB
nb_model = joblib.load('/content/drive/MyDrive/Universidad/NLP/Proyecto Final/Prueba de modelos/Modelos guardados/modelo_naive_bayes.joblib')
tfidf_vectorizer = joblib.load('/content/drive/MyDrive/Universidad/NLP/Proyecto Final/Prueba de modelos/Modelos guardados/vectorizador_tfidf.joblib')

# Cargar modelo de LSTM
# Registrar la capa de atención personalizada y cargar el modelo LSTM
with tf.keras.utils.custom_object_scope({'AttentionLayer': AttentionLayer}):
    lstm_model = tf.keras.models.load_model('/content/drive/MyDrive/Universidad/NLP/Proyecto Final/Prueba de modelos/Modelos guardados/LSTM + GloVe + FasText + CA.h5')
with open('/content/drive/MyDrive/Universidad/NLP/Proyecto Final/Prueba de modelos/Modelos guardados/tokenizer_glove.pkl', 'rb') as file:
    tokenizer = pickle.load(file)

In [5]:
# Parámetros del modelo LSTM
MAX_SEQUENCE_LENGTH = 100

# Inicializar el analizador de entimientos
sentiment_analyzer = SentimentIntensityAnalyzer()

# Preprocesamiento
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [6]:
def preprocess_text(text, lemmatize=True):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    words = [word for word in text.split() if word not in stop_words]
    if lemmatize:
        words = [lemmatizer.lemmatize(word) for word in words]
    return ' '.join(words)

def classify_comment(comment):
    # Preprocesar el comentario
    comment_processed = preprocess_text(comment)

    # 1. Probabilidad de toxicidad del modelo Naive Bayes
    comment_tfidf = tfidf_vectorizer.transform([comment_processed])
    nb_proba_toxic = nb_model.predict_proba(comment_tfidf)[0][1]  # Probabilidad de clase toxica (indice 1)

    # 2. Probabilidad de toxicidad del modelo LSTM
    comment_sequence = tokenizer.texts_to_sequences([comment_processed])
    comment_padded = pad_sequences(comment_sequence, maxlen=MAX_SEQUENCE_LENGTH)
    lstm_proba_toxic = lstm_model.predict(comment_padded)[0][0]  # La salida es una probabilidad entre 0 y 1

    # 3. Análisis de sentimientos con ajuste de compound
    sentiment_score = sentiment_analyzer.polarity_scores(comment)
    neg_score = sentiment_score['neg']
    compound_score = sentiment_score['compound']

    # Ajustar probabilidad de toxicidad del análisis de sentimientos
    if compound_score < -0.5:
        sentiment_proba_toxic = min(1.0, neg_score * 1.5)  # Refuerza la probabilidad si compound es muy negativo
    else:
        sentiment_proba_toxic = neg_score  # Usa `neg` directamente si el tono no es fuertemente negativo

    # Combinar probabilidades con los pesos especificados
    weighted_proba_toxic = (0.35 * nb_proba_toxic) + (0.40 * lstm_proba_toxic) + (0.25 * sentiment_proba_toxic)

    # Clasificar en No Tóxico, Posible Tóxico o Tóxico
    if weighted_proba_toxic <= 0.45:
        category = "No Tóxico"
    elif 0.46 <= weighted_proba_toxic <= 0.65:
        category = "Posible Tóxico"
    else:
        category = "Tóxico"

    print(f"Comentario: {comment}")
    print(f"Probabilidad de Toxicidad (Naive Bayes): {nb_proba_toxic:.2f}")
    print(f"Probabilidad de Toxicidad (LSTM): {lstm_proba_toxic:.2f}")
    print(f"Probabilidad de Toxicidad (Análisis de Sentimientos): {sentiment_proba_toxic:.2f} (compound: {compound_score:.2f})")
    print(f"Probabilidad de Toxicidad Final (Ponderada): {weighted_proba_toxic:.2f} -> {category}\n")

    return category

In [18]:
# Inicializar contadores de cada categoría
no_toxico_count = 0
posible_toxico_count = 0
toxico_count = 0

# Clasificar cada comentario y acumular los resultados
for comentario in toxico_comentario_2:
    categoria = classify_comment(comentario)
    if categoria == "No Tóxico":
        no_toxico_count += 1
    elif categoria == "Posible Tóxico":
        posible_toxico_count += 1
    elif categoria == "Tóxico":
        toxico_count += 1

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Comentario: You're so dumb, it's actually funny.
Probabilidad de Toxicidad (Naive Bayes): 0.96
Probabilidad de Toxicidad (LSTM): 1.00
Probabilidad de Toxicidad (Análisis de Sentimientos): 0.34 (compound: -0.18)
Probabilidad de Toxicidad Final (Ponderada): 0.82 -> Tóxico

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Comentario: No one cares about your opinion. Just shut up already.
Probabilidad de Toxicidad (Naive Bayes): 0.83
Probabilidad de Toxicidad (LSTM): 0.99
Probabilidad de Toxicidad (Análisis de Sentimientos): 0.17 (compound: 0.20)
Probabilidad de Toxicidad Final (Ponderada): 0.73 -> Tóxico

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Comentario: What a joke! This is literally the worst thing I've seen.
Probabilidad de Toxicidad (Naive Bayes): 0.72
Probabilidad de Toxicidad (LSTM): 0.99
Probabilidad de Toxicidad (Análisis de Sentimientos): 0.30 (compound: -0.49)
Probabilidad de Toxicidad Final (Ponderada): 0.72 -> Tóxico

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms

In [19]:
# Resumen de resultados
print("\nResumen de Categorías:")
print(f"No Tóxico: {no_toxico_count}")
print(f"Posible Tóxico: {posible_toxico_count}")
print(f"Tóxico: {toxico_count}")
# print(f"Porcentaje Tóxico: {(toxico_count / len(nuevo_comentario)) * 100:.2f}%")


Resumen de Categorías:
No Tóxico: 6
Posible Tóxico: 7
Tóxico: 38


### Comentarios

In [15]:
notoxico_comentario = [
    "I appreciate your input! Let’s see how we can build on it.",
    "Thank you for sharing your perspective; it's great to hear different ideas.",
    "This is an interesting point. I hadn’t thought of it that way!",
    "I think there’s value in what you’re saying. Let’s explore it further.",
    "That’s a solid point! I’d love to learn more about your approach.",
    "Thanks for contributing! It adds depth to the discussion.",
    "I see where you’re coming from, and it’s a refreshing take.",
    "I can tell you put thought into this. Thank you!",
    "This definitely gives us something to think about.",
    "I appreciate your dedication to this. Let’s keep it up!",
    "Good job! Let’s keep moving forward with these ideas.",
    "You bring up a good point. Let’s discuss it further.",
    "I really appreciate your effort; it’s clear you care about this.",
    "Great job articulating your thoughts! They add value to the topic.",
    "This is insightful. I’m glad you shared it with us.",
    "Your input is helping the team. Thank you!",
    "Let’s keep collaborating. Your ideas bring a lot to the table.",
    "Thank you for your feedback. It helps us see other perspectives.",
    "I like your approach. It’s refreshing and adds balance.",
    "I think we can combine our ideas to make this even better.",
    "Thank you for taking the time to contribute to this discussion.",
    "This is a good foundation! Let’s build on it.",
    "Your point of view is appreciated. Thanks for sharing!",
    "This was well thought out. It’s helpful to have this input.",
    "Your insights are very valuable. Thank you for sharing them!",
    "Your approach makes sense. Let’s see how it fits in.",
    "I like that you’re open to discussing different ideas.",
    "Your suggestion gives us more options to consider.",
    "This is a great contribution to our discussion.",
    "I think your ideas can help us improve this further.",
    "I respect your thoughts, and they add to our understanding.",
    "This is a thoughtful take. Thank you for adding it!",
    "Your perspective gives us a new angle to explore.",
    "Thanks for helping us keep an open mind about this.",
    "I can see where you're coming from; good points!",
    "Thank you for your positive approach; it’s helpful.",
    "Your insights are very constructive!",
    "Thanks for bringing new ideas to the table.",
    "That’s a very respectful and thoughtful answer.",
    "Great to have this input—it really enriches the topic.",
    "Thank you for this! It’s refreshing to hear new views.",
    "You did a good job explaining your perspective.",
    "Your contributions always help us think outside the box.",
    "I admire how you consider every angle.",
    "Thanks for approaching this with an open mind.",
    "Your approach is thoughtful and respectful. Thank you.",
    "This is a great example of teamwork. Thanks for helping!",
    "Your response helps the discussion stay balanced.",
    "It’s awesome to have diverse thoughts like yours here!",
    "Your comment is a great addition to our conversation."
]

notoxico_comentario_2 = [
    "I appreciate your input! Let’s see how we can build on it.",
    "Thank you for sharing your perspective; it's great to hear different ideas.",
    "This is an interesting point. I hadn’t thought of it that way!",
    "I think there’s value in what you’re saying. Let’s explore it further.",
    "That’s a solid point! I’d love to learn more about your approach.",
    "Thanks for contributing! It adds depth to the discussion.",
    "I see where you’re coming from, and it’s a refreshing take.",
    "I can tell you put thought into this. Thank you!",
    "This definitely gives us something to think about.",
    "I appreciate your dedication to this. Let’s keep it up!",
    "Good job! Let’s keep moving forward with these ideas.",
    "You bring up a good point. Let’s discuss it further.",
    "I really appreciate your effort; it’s clear you care about this.",
    "Great job articulating your thoughts! They add value to the topic.",
    "This is insightful. I’m glad you shared it with us.",
    "Your input is helping the team. Thank you!",
    "Let’s keep collaborating. Your ideas bring a lot to the table.",
    "Thank you for your feedback. It helps us see other perspectives.",
    "I like your approach. It’s refreshing and adds balance.",
    "I think we can combine our ideas to make this even better.",
    "Thank you for taking the time to contribute to this discussion.",
    "This is a good foundation! Let’s build on it.",
    "Your point of view is appreciated. Thanks for sharing!",
    "This was well thought out. It’s helpful to have this input.",
    "Your insights are very valuable. Thank you for sharing them!",
    "Your approach makes sense. Let’s see how it fits in.",
    "I like that you’re open to discussing different ideas.",
    "Your suggestion gives us more options to consider.",
    "This is a great contribution to our discussion.",
    "I think your ideas can help us improve this further.",
    "I respect your thoughts, and they add to our understanding.",
    "This is a thoughtful take. Thank you for adding it!",
    "Your perspective gives us a new angle to explore.",
    "Thanks for helping us keep an open mind about this.",
    "I can see where you're coming from; good points!",
    "Thank you for your positive approach; it’s helpful.",
    "Your insights are very constructive!",
    "Thanks for bringing new ideas to the table.",
    "That’s a very respectful and thoughtful answer.",
    "Great to have this input—it really enriches the topic.",
    "Thank you for this! It’s refreshing to hear new views.",
    "You did a good job explaining your perspective.",
    "Your contributions always help us think outside the box.",
    "I admire how you consider every angle.",
    "Thanks for approaching this with an open mind.",
    "Your approach is thoughtful and respectful. Thank you.",
    "This is a great example of teamwork. Thanks for helping!",
    "Your response helps the discussion stay balanced.",
    "It’s awesome to have diverse thoughts like yours here!",
    "Your comment is a great addition to our conversation."
]

In [14]:
toxico_comentario = [
    "You're a complete idiot. Nobody cares about your dumb opinions.",
    "Do you even have a brain? You're so unbelievably stupid.",
    "Honestly, you're just an annoying little piece of trash.",
    "Every single thing you say is as dumb as you are.",
    "You're a waste of space and air. Just shut up already.",
    "Get lost, loser. Nobody wants to hear from you.",
    "You sound so ignorant, it's actually painful to read.",
    "Pathetic. Only an idiot like you would think this makes sense.",
    "You're such a clueless jerk. Why even bother?",
    "Just shut up, moron. Nobody asked for your opinion.",
    "You're an absolute waste of time and everyone knows it.",
    "Everyone would be happier if you just disappeared.",
    "You’re such a know-it-all, but you know absolutely nothing.",
    "You’re the most annoying person here, without a doubt.",
    "You're such a useless, dumb person. Go away.",
    "You don't have a single clue, do you? Just pathetic.",
    "I’d be embarrassed to be as dumb as you are.",
    "Get your head out of your ass and try thinking for once.",
    "You are one of the most useless people I've ever met.",
    "Everything you say is just pure garbage.",
    "You're a joke. Nobody respects you because you're an idiot.",
    "Every time you speak, I lose brain cells.",
    "Do you enjoy being this stupid? Or does it come naturally?",
    "Your existence makes the world a worse place.",
    "If stupidity was a profession, you'd be a billionaire.",
    "I pity anyone who has to deal with you. You're the worst.",
    "Shut up already, you useless waste of time.",
    "What kind of idiot says things like this? Oh wait, you do.",
    "You’re just an arrogant fool who doesn’t know anything.",
    "You're literally the dumbest person here. It's sad.",
    "Your ideas are trash, just like everything about you.",
    "No one cares about your pathetic, useless opinions.",
    "If you left, nobody would miss you. That's a fact.",
    "Please, for the love of all things, stop being such an idiot.",
    "Nobody likes you, and for good reason. You're awful.",
    "You're just a loudmouth with no brain. Go away.",
    "You’re the reason people hate interacting online.",
    "You're so dense, it's like talking to a wall.",
    "You're a miserable excuse for a human being.",
    "Literally everything you say is the definition of stupidity.",
    "I don’t know how someone can be as clueless as you.",
    "Shut your mouth, you obnoxious jerk.",
    "You are beyond help. So dumb, it’s actually painful.",
    "The world would be better without people like you in it.",
    "Go crawl back to whatever hole you came from.",
    "You're like a disease. Everything you touch gets worse.",
    "Just leave. You're making everyone around you dumber.",
    "This is peak stupidity, but I guess that’s your specialty.",
    "Stop acting like you know anything; you’re a clueless idiot.",
    "I’d rather talk to a rock than listen to your nonsense.",
    "You're just a toxic leech. Nobody needs you here."
]

toxico_comentario_2 = [
    "You're so dumb, it's actually funny.",
    "No one cares about your opinion. Just shut up already.",
    "What a joke! This is literally the worst thing I've seen.",
    "Please stop. You're completely useless.",
    "Nobody asked for your input, genius.",
    "You clearly don’t know what you’re doing. Why even try?",
    "The fact that you thought this was a good idea is pathetic.",
    "You're an embarrassment. Honestly, give up.",
    "This is just trash. No one with any sense would agree with you.",
    "Why are you even here? You’re just making things worse.",
    "This is garbage. You have no idea what you’re talking about.",
    "If I had a dollar for every dumb thing you've said, I'd be rich.",
    "Stop trying. Everyone can see you're failing.",
    "Wow, how can someone be this clueless?",
    "You're so annoying. No one cares about anything you say.",
    "You are the definition of failure. Just look at this mess.",
    "You clearly don’t have a brain.",
    "Pathetic! Do everyone a favor and stay quiet.",
    "This is painfully bad. Stop embarrassing yourself.",
    "Please, spare us from your nonsense.",
    "You think you’re smart, but you’re actually clueless.",
    "You're the worst at this. Nobody wants you here.",
    "Are you for real? You can’t be that dumb.",
    "It’s embarrassing watching you try so hard and fail.",
    "You’re so bad at this it’s almost sad.",
    "Why are you even trying? You have no talent.",
    "This is cringe-worthy. Please go away.",
    "Get a life, seriously. You're a joke.",
    "Do us a favor and stop posting. No one cares.",
    "You’re so dumb, I can’t even believe it.",
    "Pathetic attempt, as usual.",
    "I can’t believe people like you actually exist.",
    "Stop wasting everyone’s time with your nonsense.",
    "You’ve got to be kidding. This is horrendous.",
    "This is laughably terrible.",
    "You're a failure, plain and simple.",
    "You're embarrassingly bad at this.",
    "Nobody cares what you think, loser.",
    "How can you be this bad? It’s painful to watch.",
    "What a waste of time. You add nothing of value.",
    "Please, you’re way out of your league here.",
    "Wow, you're even worse than I thought.",
    "Stop talking, you’re just making it worse.",
    "You’re so clueless it’s actually painful.",
    "Every word you say is dumber than the last.",
    "Honestly, you’re a joke. Just quit already.",
    "Do you ever think before you talk? Because wow...",
    "Are you trying to be this dumb, or is it natural?",
    "Why don’t you just leave? No one wants you here.",
    "You're not only wrong but also incredibly annoying.",
    "You might be the worst person to attempt this. Ever."
]